# TOY PROBLEM

The purpose of this notebook is to illustrate the impact that a classifier can have on the world. 

The toy example is as follows:

In a magical land, there's 100 blue haired people and 100 green haired people. Every day, they can eat normal food, or they can steal a cookie from a sacred cookie jar in their house. The initial probabilities of stealing a cookie are $\mu_{blue}=0.6$ for the blues and $\mu_{green}=0.4$ for the greens, as a population. There is also a central fucked up dystopian government, which sends police to the houses of people that it thinks are likely to have stolen a cookie. If the police catches someone stealing a cookie, that person's food is confiscated for the day, and they go hungry (which means that they're likelier to want a cookie the next day, because cookies are more filling than normal food). If not, they progressively lose interest in stealing the cookies, and their probabilities of stealing a cookie diminishes. 

## More formally

Each individual is associated with a feature vector $X$, which is made up of different distributions with mean $\mu$.
At each time step, the means mu are used to generate actual stealing data, i.e. to determine if an individual stole a cookie or not, represented by a vector $y$ full of zeros and ones.
This data is fed into a logistic regression classifier.
Each time step, and for each individual, the classifier predicts whether or not someone will steal a cookie, and depending on that prediction, will decide to send or not send a policeman to their house. If the police is sent to someone's house, and that person gets caught stealing a cookie, that person's $\mu$ increases by $\alpha$. Otherwise, it decreases by $\beta$.

Then I plot each person's $\mu$ over time to see how it evolves!

In [66]:
#import things
import numpy as np
import sklearn 
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import matplotlib.pyplot as plt
np.set_printoptions(precision=3, suppress=True)

#set up hyperparameters
#initial probability that blue and green steal
mu_blue = 0.65
mu_green = 0.35
#number of people in each population
N = 100
#probability increase/decrease if caught/not caught
alpha = 0.003
beta = 0.003

In [67]:
#generate random uniforms between 0 and 1
y_blue = np.random.uniform(0,1, (N,1))
y_green = np.random.uniform(0,1, (N,1))

#replace each value with 0 or 1
y_blue[y_blue > mu_blue] = 1
y_blue[y_blue <= mu_blue] = 0
y_green[y_green > mu_green] = 1
y_green[y_green <= mu_green] = 0

y_green = np.abs(y_green - 1)
y_blue = np.abs(y_blue - 1)

Now, the goal is to make feature vectors for blue and green. This is gonna be simpler than last time: there'll only be a single feature, which is a noisy estimate of the true probability. 
We do that, and make a historical X and y, to make a preliminary model, and then also make a real X.

In [68]:
#create identities for each being:
    #give them a number
identities = np.arange(2*N)
identities = np.reshape(identities,(200,1))
    #give them a base rate of stealing the cookies
base_rates_blue = np.ones((N, 1))*mu_blue
base_rates_green = np.ones((N, 1))*mu_green
base_rates = np.concatenate((base_rates_blue, base_rates_green), axis=0)

#create X_blue
X_blue= np.random.normal(mu_blue, 0.001, (N,1))

#create X_green
X_green= np.random.normal(mu_green, 0.001, (N,1))

#append identities, base rates, and mix everything together
big_X = np.concatenate((X_blue, X_green), axis=0)
big_X = np.concatenate((big_X, identities, base_rates), axis=1)
np.random.shuffle(big_X)


X = np.copy(big_X[:,0])
identities = big_X[:,1]
base_rates = big_X[:,2]

historical_X_blue = np.concatenate((X_blue, y_blue), axis = 1)
historical_X_green = np.concatenate((X_green, y_green), axis = 1)
historical_X = np.concatenate((historical_X_blue, historical_X_green), axis=0)
np.random.shuffle(historical_X)

historical_y = historical_X[:,1]
historical_X = historical_X[:,0]

Quickly train model by making up a y

In [72]:
# instantiate a logistic regression model, and fit with X and y
model = LogisticRegression()
model = model.fit(historical_X.reshape(-1,1), historical_y)

# check the accuracy on the training set
model.score(historical_X.reshape(-1, 1), historical_y)

0.67

Hella functions:

In [73]:
#function to generate actual stealing or not stealing events
def did_they_steal(base_rates):
    
    y = np.zeros((len(base_rates),1))
    
    for i in range(len(base_rates)):
        if base_rates[i] > np.random.uniform(0,1):
            y[i] = 1
    
    return y

#function to update base rates
def update_base_rates(base_rates, y, predictions, alpha, beta):
    for i in range(len(y)):
        if y[i] == predictions[i] and y[i] == 1 and (base_rates[i] + beta)  < 1:
            base_rates[i] = base_rates[i] + beta
        elif base_rates[i] > alpha:
            base_rates[i] = base_rates[i] - alpha
            
    return base_rates

#function that uses new base rates to create new feature vector
def update_X(base_rates):
    for i in range(len(base_rates)):
        new_vec1 = np.random.normal(base_rates[i]*3, 1, (1, 3))
        new_vec2 = np.random.exponential(base_rates[i]+1, (1, 3))
        new_vec3 = np.random.binomial(100, base_rates[i]**2, (1, 3))
        new_vec4 = np.random.standard_normal((1, 1))
        new_features = (new_vec1, new_vec2, new_vec3, new_vec4)

        X[i] = np.concatenate(new_features, axis=1)
    
    return X

#predicts probability that someone stole
def predict_probs_steal(X, modelcoef, modelintercept):
    predictions = 1 / (1 + np.exp(-(X @ np.transpose(modelcoef) + modelintercept)))
    predictions[predictions > 0.5] = 1
    predictions[predictions <= 0.5] = 0
    
    return predictions

In [ ]:
y